In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import lightgbm as lgb

# contest_basic_train表的数据分析

In [2]:
train=pd.read_csv('data/contest_basic_train.tsv', sep='\t', header=0)

In [3]:
train.head()

,REPORT_ID,ID_CARD,LOAN_DATE,AGENT,IS_LOCAL,WORK_PROVINCE,EDU_LEVEL,MARRY_STATUS,SALARY,HAS_FUND,Y
0,8787,320382**********13,2017/2/15,APP,本地籍,320000.0,本科,已婚,3.0,0.0,0
1,9410,120223**********8X,2017/3/13,wechat,本地籍,120000.0,NaN,已婚,6.0,1.0,0
2,22999,330226**********90,2017/2/27,APP,本地籍,NaN,专科及以下,已婚,4.0,0.0,0
3,24820,230105**********20,2017/2/24,APP,本地籍,230000.0,专科及以下,已婚,2.0,0.0,1
4,25708,340222**********29,2017/1/16,NaN,本地籍,NaN,专科,未婚,NaN,1.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
REPORT_ID        30000 non-null int64
ID_CARD          30000 non-null object
LOAN_DATE        30000 non-null object
AGENT            8952 non-null object
IS_LOCAL         30000 non-null object
WORK_PROVINCE    27742 non-null float64
EDU_LEVEL        26942 non-null object
MARRY_STATUS     30000 non-null object
SALARY           8864 non-null float64
HAS_FUND         29998 non-null float64
Y                30000 non-null int64
dtypes: float64(3), int64(2), object(6)
memory usage: 2.5+ MB


In [5]:
train['EDU_LEVEL'].value_counts(dropna=False)# 缺失值按专科补

专科       7997
本科       6536
高中       6062
专科及以下    4452
NaN      3058
初中       1355
其他        301
硕士研究生     153
博士研究生      51
硕士及以上      35
Name: EDU_LEVEL, dtype: int64

In [6]:
train['EDU_LEVEL']=train['EDU_LEVEL'].fillna('专科')

In [7]:
train['HAS_FUND'].value_counts(dropna=False)

 0.0    16338
 1.0    13660
NaN         2
Name: HAS_FUND, dtype: int64

In [8]:
train['IS_LOCAL'].value_counts(dropna=False)

本地籍     17506
非本地籍    12494
Name: IS_LOCAL, dtype: int64

In [9]:
train['AGENT'].value_counts(dropna=False)#缺失值太多，删除

NaN           21048
APP            6644
wechat         1339
chinapnr        332
rongzhijia      299
huifusdb        240
fenqile          38
ifpp             30
kuaiqian         17
bestpay           8
orgloan           3
weijinhui         2
Name: AGENT, dtype: int64

In [10]:
train=train.drop(['AGENT'],axis=1)

In [11]:
train['MARRY_STATUS'].value_counts(dropna=False)

已婚    19199
未婚     9566
离异      719
离婚      314
其他      161
丧偶       41
Name: MARRY_STATUS, dtype: int64

In [12]:
train['SALARY'].value_counts(dropna=False)#缺失值太多，删除

NaN     21136
 3.0     3057
 4.0     2117
 2.0     1766
 5.0      753
 7.0      599
 6.0      475
 1.0       97
Name: SALARY, dtype: int64

In [13]:
train=train.drop(['SALARY'],axis=1)

In [14]:
train['Y'].value_counts(dropna=False)

0    28125
1     1875
Name: Y, dtype: int64

In [15]:
# One-hot
dumm_edu_level= pd.get_dummies(train['EDU_LEVEL'], prefix='EDU_LEVEL')
final_train = train.join(dumm_edu_level)
final_train=final_train.drop(['EDU_LEVEL'],axis=1)
dumm_marry= pd.get_dummies(train['MARRY_STATUS'], prefix='MARRY_STATUS')
final_train = final_train.join(dumm_marry)
final_train=final_train.drop(['MARRY_STATUS'],axis=1)
dumm_is_local = pd.get_dummies(train['IS_LOCAL'], prefix='IS_LOCAL')
final_train = final_train.join(dumm_is_local)
final_train=final_train.drop(['IS_LOCAL'],axis=1)
dumm_has_fund = pd.get_dummies(train['HAS_FUND'], prefix='HAS_FUND')
final_train = final_train.join(dumm_has_fund)
final_train=final_train.drop(['HAS_FUND'],axis=1)

In [16]:
final_train=final_train.drop(['REPORT_ID','ID_CARD','LOAN_DATE','WORK_PROVINCE'],axis=1)

In [17]:
final_train

,Y,EDU_LEVEL_专科,EDU_LEVEL_专科及以下,EDU_LEVEL_其他,EDU_LEVEL_初中,EDU_LEVEL_博士研究生,EDU_LEVEL_本科,EDU_LEVEL_硕士及以上,EDU_LEVEL_硕士研究生,EDU_LEVEL_高中,MARRY_STATUS_丧偶,MARRY_STATUS_其他,MARRY_STATUS_已婚,MARRY_STATUS_未婚,MARRY_STATUS_离婚,MARRY_STATUS_离异,IS_LOCAL_本地籍,IS_LOCAL_非本地籍,HAS_FUND_0.0,HAS_FUND_1.0
0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0
1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
2,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
3,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
5,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
6,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
7,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0
8,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
9,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1


In [18]:
final_train=final_train.sample(frac=1,random_state=100) 

In [19]:
train_y,test_y = final_train['Y'].values[:24000],final_train['Y'].values[24000:]

In [20]:
final_train=final_train.drop(['Y'],axis=1)

In [21]:
train_x,test_x = final_train[:24000].values,final_train[24000:].values

In [22]:
train_x

array([[1, 0, 0, ..., 1, 0, 1],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ..., 
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 1]], dtype=uint8)

In [23]:
test_x

array([[1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 1, 0]], dtype=uint8)

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [25]:
model = LinearSVC()
model.fit(train_x, train_y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [26]:
y_pre=model.predict(test_x)

In [27]:
accuracy_score(test_y,y_pre)

0.93700000000000006

In [28]:
np.array(test_y==1).sum()

378

In [29]:
(6000-378)/6000

0.937

In [30]:
from sklearn import svm
clf = svm.SVC()
model.fit(train_x, train_y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [31]:
y_pre=model.predict(test_x)
accuracy_score(test_y,y_pre)

0.93700000000000006

In [32]:
y_pre_train=model.predict(train_x)
accuracy_score(train_y,y_pre_train)

0.93762500000000004

In [33]:
np.array(train_y==1).sum()

1497

In [34]:
(24000-1497)/24000

0.937625